In [54]:
import pandas as pd
import geopandas as gpd
import fiona

For Edmonton, there's geospatial data on the open data site. There's a cetegory here: https://data.edmonton.ca/browse?category=Geospatial+Boundaries 

I got the general process from here: https://towardsdatascience.com/how-to-easily-join-data-by-location-in-python-spatial-join-197490ff3544

You may be able to search for specifics. 

In [55]:
#Load some house data
df = pd.read_csv("Assessments.csv")
df = df.drop(columns={"Neighbourhood"})
df.head()

,Account Number,Suite,House Number,Street Name,Garage,Neighbourhood ID,Ward,Assessed Value,Latitude,Longitude
0,1066158,NaN,14904.0,167 AVENUE NW,N,NaN,NaN,86000,53.630497,-113.580474
1,10508233,NaN,16340.0,MARK MESSIER TRAIL NW,N,NaN,NaN,45000,53.617335,-113.605690
2,1194398,NaN,15617.0,83 STREET NW,N,2050.0,Ward 3,285000,53.617007,-113.467976
3,1034214,NaN,10661.0,161 AVENUE NW,Y,3040.0,Ward 3,635000,53.619978,-113.506990
4,1114701,NaN,15003.0,76 AVENUE NW,Y,4430.0,Ward 5,407500,53.508644,-113.579897


In [56]:
#Make a geo friendly thing for the above data. 
ass = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
ass.head()

,Account Number,Suite,House Number,Street Name,Garage,Neighbourhood ID,Ward,Assessed Value,Latitude,Longitude,geometry
0,1066158,NaN,14904.0,167 AVENUE NW,N,NaN,NaN,86000,53.630497,-113.580474,POINT (-113.58047 53.63050)
1,10508233,NaN,16340.0,MARK MESSIER TRAIL NW,N,NaN,NaN,45000,53.617335,-113.605690,POINT (-113.60569 53.61733)
2,1194398,NaN,15617.0,83 STREET NW,N,2050.0,Ward 3,285000,53.617007,-113.467976,POINT (-113.46798 53.61701)
3,1034214,NaN,10661.0,161 AVENUE NW,Y,3040.0,Ward 3,635000,53.619978,-113.506990,POINT (-113.50699 53.61998)
4,1114701,NaN,15003.0,76 AVENUE NW,Y,4430.0,Ward 5,407500,53.508644,-113.579897,POINT (-113.57990 53.50864)


In [57]:
#Load some geospatial data for 'hoods
#https://data.edmonton.ca/City-Administration/City-of-Edmonton-Neighbourhoods/65fr-66s6
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
neighborhoods = gpd.read_file("edmontonHoods.kml", driver='KML')
neighborhoods.head()

,Name,Description,geometry
0,ABBOTTSFIELD,"Abbottsfield was once owned by E.F. Carey, one...","MULTIPOLYGON (((-113.38763 53.57613, -113.3888..."
1,ALBANY,,"MULTIPOLYGON (((-113.56025 53.63160, -113.5622..."
2,ALBERTA AVENUE,One of the city's older residential neighbourh...,"MULTIPOLYGON (((-113.49226 53.57662, -113.4922..."
3,ALBERTA PARK INDUSTRIAL,This non-residential neighbourhood is largely ...,"MULTIPOLYGON (((-113.59041 53.56654, -113.5904..."
4,ALDERGROVE,"Planned in the 1970s, this area's multi-family...","MULTIPOLYGON (((-113.65209 53.51950, -113.6520..."


In [58]:
#Do a join. Which houses are within which neighborhood
#joined = gpd.sjoin(ass, neighborhoods, op="within")
joined = ass.sjoin(neighborhoods, how="inner", predicate='within')
joined.head()

/Users/akeems/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


,Account Number,Suite,House Number,Street Name,Garage,Neighbourhood ID,Ward,Assessed Value,Latitude,Longitude,geometry,index_right,Name,Description
2,1194398,NaN,15617.0,83 STREET NW,N,2050.0,Ward 3,285000,53.617007,-113.467976,POINT (-113.46798 53.61701),36,BELLE RIVE,
137,1195619,NaN,8316.0,157 AVENUE NW,N,2050.0,Ward 3,264000,53.617972,-113.469187,POINT (-113.46919 53.61797),36,BELLE RIVE,
197,1195858,NaN,8305.0,157 AVENUE NW,Y,2050.0,Ward 3,260500,53.617503,-113.468711,POINT (-113.46871 53.61750),36,BELLE RIVE,
337,1196229,NaN,15612.0,83A STREET NW,N,2050.0,Ward 3,239000,53.616941,-113.470320,POINT (-113.47032 53.61694),36,BELLE RIVE,
351,1196815,NaN,8410.0,156 AVENUE NW,N,2050.0,Ward 3,267500,53.616797,-113.472399,POINT (-113.47240 53.61680),36,BELLE RIVE,


In [61]:
#In general, to join other data
df2 = pd.read_csv("edmontonIncome.csv")
df2 = df2.rename(columns={"Neighbourhood Number":"Neighbourhood ID"})
df2.head()

FileNotFoundError: [Errno 2] No such file or directory: 'edmontonIncome.csv'

In [60]:
#Join
final = pd.merge(joined, df2, on="Neighbourhood ID")
final.sample(10)

,Account Number,Suite,House Number,Street Name,Garage,Neighbourhood ID,Ward_x,Assessed Value,Latitude,Longitude,...,Neighbourhood Name,"Less than $30,000","$30,000 to less than $60,000","$60,000 to less than $100,000","$100,000 to less than $125,000","$125,000 to less than $150,000","$150,000 to less than $200,000","$200,000 to less than $250,000","$250,000 or more",No Response
176874,6382592,NaN,8907.0,129B AVENUE NW,Y,2420.0,Ward 7,277500,53.588081,-113.479198,...,KILLARNEY,179,284,216,64,29,11,4,2,905
45711,10701985,304,10811.0,72 AVENUE NW,N,5330.0,Ward 8,284500,53.508201,-113.510385,...,QUEEN ALEXANDRA,365,465,427,175,85,72,21,19,1261
47525,1280072,NaN,9502.0,154 STREET NW,Y,4580.0,Ward 1,362000,53.531686,-113.587303,...,WEST JASPER PLACE,169,309,83,27,22,14,7,4,839
164235,10136580,NaN,12303.0,89 STREET NW,Y,1080.0,Ward 7,273000,53.578435,-113.477472,...,DELTON,145,155,149,42,26,23,6,3,298
304375,10094262,121,612.0,111 STREET SW,N,5452.0,Ward 9,235000,53.426953,-113.518199,...,MACEWAN,69,274,360,187,128,104,37,11,1064
76847,1008606,NaN,8724.0,152B AVENUE NW,Y,2260.0,Ward 3,314500,53.614743,-113.475795,...,EVANSDALE,298,460,223,75,44,26,11,5,974
200767,9413014,201,996.0,KNOTTWOOD ROAD SOUTH NW,N,6450.0,Ward 11,118500,53.442472,-113.444084,...,MENISA,235,186,175,80,51,40,13,5,140
334042,10764161,NaN,7819.0,174 AVENUE NW,N,2700.0,Ward 3,157000,53.634486,-113.464466,...,SCHONSEE,91,114,179,96,56,79,31,25,345
58104,6924120,NaN,16720.0,110 STREET NW,Y,3030.0,Ward 3,371000,53.627327,-113.513909,...,BATURYN,33,138,202,101,46,26,7,4,1191
96189,4243499,NaN,706.0,ORMSBY ROAD WEST NW,Y,4380.0,Ward 5,402500,53.497810,-113.653585,...,ORMSBY PLACE,107,216,246,117,96,97,41,34,946
